## Ejercicio 7

Utilizando los atributos numéricos del archivo Autos.csv entrene un combinador lineal para predecir el
precio del auto en función del resto de los atributos. Tenga en cuenta la existencia de valores faltantes.
Realice varias ejecuciones y observe los vectores de pesos obtenidos. ¿Puede identificar cuáles son los
atributos con mayor incidencia en la determinación del precio? Explique.

In [30]:
import pandas as pd

ALPHA	= 0.01
MAX_ITE = 1000
COTA	= 10e-6

DATOS_DIR   = '../../Datos/'
data = pd.read_csv(DATOS_DIR + 'autos.csv')

##### Preprocesamiento de los datos

In [33]:
import numpy as np
data.dtypes
data["price"] = pd.to_numeric(data["price"], errors="coerce")
numeric_data = data.select_dtypes(include=[np.number])
numeric_data.isnull().sum()
numeric_data = numeric_data.fillna(numeric_data.mean())


##### Entrenamiento

In [42]:
import numpy as np
import pandas as pd
from Fuentes.ClassNeuronaGral import *
from sklearn.preprocessing import MinMaxScaler

X_raw = numeric_data.drop("price", axis=1)
T_raw = numeric_data["price"].values.reshape(-1, 1)

X_scaler, T_scaler = MinMaxScaler(), MinMaxScaler()
X = X_scaler.fit_transform(X_raw)
T = T_scaler.fit_transform(T_raw)

attributes = [col for col in numeric_data.columns if col != "price"]

n_runs = 10

weight_abs_list = []

for _ in range(n_runs):
    ng = NeuronaGradiente(alpha=ALPHA, n_iter=MAX_ITE, cotaE=COTA, FUN="linear")
    ng.fit(X, T)
    weight_abs_list.append(np.abs(ng.w_))

weight_abs_array = np.array(weight_abs_list)
weight_mean = weight_abs_array.mean(axis=0)

weight_df = pd.DataFrame({
    "Attribute": attributes,
    "Mean weight (abs)": weight_mean
})

weight_df = weight_df.sort_values(by="Mean weight (abs)", ascending=False).reset_index(drop=True)
print(weight_df)


           Attribute  Mean weight (abs)
0        engine-size           0.700918
1           city-mpg           0.150580
2        curb-weight           0.141827
3             height           0.073316
4  compression-ratio           0.071139
5        highway-mpg           0.065128
6              width           0.055743
7          symboling           0.051453
8             length           0.022567
9         wheel-base           0.014068


> **Conclusión:** Los atributos más relevantes son:
> - engine-size.
> - city-mpg.
> - curb-weight.